### Stable Diffusion WebUI Colab by @Philogicae
To use it with Photoshop, follow: https://github.com/isekaidev/stable.art#how-to-install

In [ ]:
import os
from google.colab import drive
drive_path = "/content/drive"
data_dir = "/content/drive/MyDrive/stable-diffusion"
sd_dir = "/content/stable-diffusion-webui"

# Mount Google Drive
if not os.path.exists(drive_path):
    try:
        drive.mount(drive_path, force_remount=True)
        os.makedirs(data_dir, exist_ok=True)
    except:
        raise("Error mounting drive or with path variables")

# Requirements
%pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.16/xformers-0.0.16+814314d.d20230118-cp38-cp38-linux_x86_64.whl
%pip install -q --pre triton
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git

# Extensions
%cd {sd_dir}/extensions
!git clone https://github.com/yfszzx/stable-diffusion-webui-images-browser.git
!git clone https://github.com/imrayya/stable-diffusion-webui-Prompt_Generator.git
!git clone https://github.com/philogicae/sd-civitai-browser.git
!git clone https://github.com/zero01101/openOutpaint-webUI-extension.git
!git clone https://github.com/deforum-art/deforum-for-automatic1111-webui.git

# Scripts
%cd {sd_dir}/scripts
!wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py

# Embeddings
%cd {sd_dir}/embeddings
!wget https://huggingface.co/joachimsallstrom/Double-Exposure-Embedding/resolve/main/dblx.pt

# Outputs / Config -> Saved to Google Drive
%cd {sd_dir}
output_path = f"{data_dir}/outputs"
os.makedirs(output_path, exist_ok=True)
!rm -Rf outputs && ln -s {output_path} outputs

config_path = f"{data_dir}/config"
os.makedirs(config_path, exist_ok=True)
for filename in ["config.json", "ui-config.json", "styles.csv", "artists.csv"]:
    filepath = f"{config_path}/{filename}"
    if not os.path.exists(filepath):
        if filename.endswith(".json"):
            with open(filepath, mode="w") as f:
                f.write("{}")
        else:
            !touch {config_path}/{filename}
    !rm -f {filename} && ln -s {config_path}/{filename} {filename}

# Default Models (Uncomment to use)
%cd {sd_dir}/models/Stable-diffusion
## Stable-diffusion 2.1
!wget https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors
!wget https://huggingface.co/webui/stable-diffusion-2-1/raw/main/v2-1_768-ema-pruned.yaml
## Stable-diffusion inpainting
#!wget https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/blob/main/512-inpainting-ema.safetensors
#!wget https://huggingface.co/webui/stable-diffusion-2-inpainting/raw/main/512-inpainting-ema.yaml
## Stable-diffusion upscaler
#!wget https://huggingface.co/stabilityai/stable-diffusion-x4-upscaler/blob/main/x4-upscaler-ema.safetensors
#!wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference.yaml -O x4-upscaler-ema.yaml
## Instruct-pix2pix
#!wget https://huggingface.co/timbrooks/instruct-pix2pix/blob/main/instruct-pix2pix-00-22000.safetensors

# Load Saved Models (From GoogleDrive/stable-diffusion/models)
models_path = f"{data_dir}/models"
os.makedirs(models_path, exist_ok=True)
for filename in os.listdir(models_path):
    !ln -s {models_path}/{filename} {filename}

# Launch
%cd {sd_dir}
!python launch.py --theme dark --share --xformers --medvram --api